In [7]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [38]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('../../../baseline/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
root='../../../dataset/'

epoch = 'latest'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/2023_05_12_01_40_01_FasterRCNN_SGKF_4'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [39]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [40]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: ./work_dirs/2023_05_12_01_40_01_FasterRCNN_SGKF_4/latest.pth


In [41]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 27.7 task/s, elapsed: 176s, ETA:     0s

In [48]:
print(output[0])
print(len(output[0][1]))

[array([[6.3648590e+02, 2.5704266e+02, 7.3159052e+02, 3.3985742e+02,
        2.8084978e-01],
       [4.7216901e+02, 5.7043457e+02, 5.6394354e+02, 6.7353894e+02,
        2.4120244e-01],
       [2.4050124e+02, 6.7555621e+02, 2.9698926e+02, 7.4750165e+02,
        1.2674889e-01],
       [1.9575656e+02, 6.7436731e+02, 2.9410239e+02, 7.3003113e+02,
        9.8645650e-02],
       [4.8425940e+02, 5.9106671e+02, 5.5830811e+02, 6.4912665e+02,
        9.0141475e-02],
       [3.2937317e+02, 6.3836444e+02, 4.0353601e+02, 7.1620355e+02,
        7.2039030e-02],
       [1.2951747e+02, 2.7910858e+02, 2.3849889e+02, 3.1113397e+02,
        6.6974029e-02],
       [1.9083206e+01, 3.1299362e+02, 2.2852461e+02, 4.3921249e+02,
        5.4893676e-02],
       [3.3114999e+02, 6.1177405e+02, 4.3878574e+02, 7.2760645e+02,
        5.0063908e-02]], dtype=float32), array([[5.62395020e+02, 1.09166626e+02, 7.71359863e+02, 3.69143036e+02,
        9.86364305e-01],
       [6.21270569e+02, 1.59605225e+02, 7.42932068e+02, 3

In [42]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.28084978 636.4859 257.04266 731.5905 339.8...,test/0000.jpg
1,0 0.67504233 761.03503 662.4424 894.3799 989.8...,test/0001.jpg
2,0 0.68734664 876.7832 469.27927 1023.3169 629....,test/0002.jpg
3,0 0.40439713 132.2482 140.74684 205.32605 189....,test/0003.jpg
4,0 0.5498809 154.18903 247.15652 893.7128 790.4...,test/0004.jpg
